In [25]:
import IPython, IPython.display
import numpy, PIL
import cStringIO
import scipy.ndimage
import scipy.misc
import matplotlib
import matplotlib.image

import warnings
warnings.simplefilter("ignore", DeprecationWarning)

def fimg_ar_as_PIL(img_ar):
    return PIL.Image.fromarray(numpy.uint8(img_ar))
def fimg_PIL_as_ar(img_pil):
    return numpy.array(img_pil)
def fimg_ar_disp_IPython(img_ar):
    io = cStringIO.StringIO()
    fimg_ar_as_PIL(img_ar).save(io, 'bmp')
    IPython.display.display(IPython.display.Image(data=io.getvalue()))
def fimg_ar_disp(img_ar):
    fimg_ar_disp_IPython(img_ar)
def fimg_load_ar(img_path):
    return matplotlib.image.imread(img_path)
def fimg_ar_size(img_ar):
    return (len(img_ar[0]) if img_ar is not None and len(img_ar) else 0.0, len(img_ar) if img_ar is not None else 0.0)
def fimg_ar_racoon():
    return scipy.misc.face()
def fimg_ar_resize(img_ar, scl):
    return scipy.ndimage.zoom(img_ar, (scl[0],scl[1],1))
def fimg_filled(img_sz, color_val = 0):
    img = np.zeros([img_sz[0],img_sz[1],3],dtype=np.uint8)
    if color_val != 0:
        img.fill(color_val)
    return img

%matplotlib inline
g_has_matplotlib_anim = matplotlib.__version__ > '1.5.0'
if g_has_matplotlib_anim:
    import matplotlib.pyplot as plt
    import matplotlib.animation
    from IPython.display import HTML
    
def fimg_animate_plt(imgs, interval=50):
    if not g_has_matplotlib_anim:
        return None
    def updatefig(frame, ctx, *args):
        ctx['img_i'] = (ctx['img_i'] + 1) % len(ctx['imgs'])
        ctx['im'].set_array(ctx['imgs'][ctx['img_i']])
        return im,
    fig = plt.figure()
    im = plt.imshow(imgs[0], animated=True)
    ctx = {'im':im, 'img_i':0, 'imgs':imgs}
    anim = matplotlib.animation.FuncAnimation(fig, updatefig, interval=interval, blit=True, frames = len(imgs), fargs=(ctx, None))
    return anim
def test_fimg_animate_plt():
    def f(x, y):
        return np.sin(x) + np.cos(y)
    x = np.linspace(0, 2 * np.pi, 120)
    y = np.linspace(0, 2 * np.pi, 100).reshape(-1, 1)
    return fimg_animate_plt( [f(x, y), f(x,y)]*20 )
#HTML(test_fimg_animate_plt().to_html5_video())

In [28]:
import numpy as np

g_pinhole_f = 0.1
g_pinhole_sensor_sz = [0.1,0.1]
g_pihole_img_sz = [256,256]

def fpinhole_matrix(focal):
    return [[-focal, 0, 0],
            [0, -focal, 0],
            [0, 0, 1]]
def fpinhole_viewp(sensor_sz, img_sz):
    return [[img_sz[0]/sensor_sz[0], 0, img_sz[0]/2.0],
            [0, -img_sz[1]/sensor_sz[1], img_sz[1]/2.0],
            [0, 0, 1]]
def fpinhole_pin_pt(pin_mat, pt):
    homog = np.dot(pin_mat, pt)
    #print pt, homog
    return [homog[0]/homog[2], homog[1]/homog[2]]
def fpinhole_viewp_pt(viewp_mat, pt):
    viewp_pt = np.dot(viewp_mat, list(pt)+[1.0])
    return viewp_pt[0:2]
def fpinhole_project_img(pin_mat, viewp_mat, pts, img, pt_colors=[255,255,255]):
    img_sz = fimg_ar_size(img)
    for i_pt, pt in enumerate(pts):
        pin_pt = fpinhole_pin_pt(pin_mat, pt)
        img_pt = [int(x+0.5) for x in fpinhole_viewp_pt(viewp_mat, pin_pt)]
        #print img_pt
        if all([img_pt[i] >= 0 and img_pt[i] < img_sz[i] for i in range(2)]):
            img[img_pt[1]][img_pt[0]] = pt_colors[i_pt if i_pt < len(pt_colors) else 0]
        else: 
            print 'XXX'
def fpinhole_good_offset(focal, sensor_sz, rad):
    return -rad - max(focal, (focal * rad / (0.5 * min(sensor_sz)))) 
def ftest_pinhole():
    pin_img = fimg_filled(g_pihole_img_sz)
    pin_mat = fpinhole_matrix(g_pinhole_f)
    viewp_mat = fpinhole_viewp(g_pinhole_sensor_sz, g_pihole_img_sz)
    f = -g_pinhole_f
    pts = [ [0, 0, 2*f], 
           [-f/2,-f/2,2*f], [-f/2,f/2,2*f] , [f/2,f/2,2*f], [f/2,-f/2,2*f], 
           [-f/2,-f/2,3*f], [-f/2,f/2,3*f] , [f/2,f/2,3*f], [f/2,-f/2,3*f]
          ]
    pt_cols = [[255,255,255], [255,0,0], [0,0,255]]
    fpinhole_project_img(pin_mat, viewp_mat, pts, pin_img, pt_cols)
    fimg_ar_disp(pin_img)
#ftest_pinhole()

In [18]:
import re

def _fread_obj_vertices_str(obj_str):
    regex = r"^v\s(-?(0|[1-9]\d*)(\.\d+)?)\s(-?(0|[1-9]\d*)(\.\d+)?)\s(-?(0|[1-9]\d*)(\.\d+)?)"
    matches = re.finditer(regex, obj_str, re.MULTILINE)
    verts = []
    for i, match in enumerate(matches):
        verts.append([float(match.groups()[i]) for i in [0,3,6]])
    return verts
def fread_obj_vertices_str(obj_str):
    regex = r"^v\s(.+)\s(.+)\s(.+)"
    matches = re.finditer(regex, obj_str, re.MULTILINE)
    verts = []
    for i, match in enumerate(matches):
        verts.append([float(match.groups()[i]) for i in [0,1,2]])
    return verts
def fread_obj_vertices_file(file):
    verts = []
    with open(file) as fi:
        for line in fi:
            verts.extend(fread_obj_vertices_str(line))
    return verts
def ftest_obj():
    print fread_obj_vertices_file('./data/suzanne.obj')
def fbox_vertices(verts):
    box_max = reduce(lambda a,b: np.maximum(a,b), verts)
    box_min = reduce(lambda a,b: np.minimum(a,b), verts)
    return box_min, box_max
def fcenter_vertices(verts):
    box_min, box_max = fbox_vertices(verts)
    box_cent = 0.5*np.add(box_max, box_min)
    return [np.subtract(x, box_cent) for x in verts]
def frad_vertices(verts):
    box_min, box_max = fbox_vertices(verts)
    return max(np.maximum(np.absolute(box_min), np.absolute(box_max)))
def fpinhole_adjust_vertices(focal, sensor_sz, verts, adjust_zoom = 1.0):
    verts = fcenter_vertices(verts)
    off_z = fpinhole_good_offset(focal, sensor_sz, adjust_zoom * frad_vertices(verts))
    return [x + [0.0, 0.0, off_z] for x in verts]
def frender_verts_img(verts, adjust = False, adjust_zoom = 1.0, focal = g_pinhole_f, sensor_sz = g_pinhole_sensor_sz, img_sz = g_pihole_img_sz):
    pin_img = fimg_filled(img_sz)
    pin_mat = fpinhole_matrix(focal)
    viewp_mat = fpinhole_viewp(sensor_sz, img_sz)
    if adjust:
        verts = fpinhole_adjust_vertices(focal, sensor_sz, verts, adjust_zoom)
    fpinhole_project_img(pin_mat, viewp_mat, verts, pin_img)
    return pin_img
def frender_model_img(filename = './data/bunny.obj', adjust = True, adjust_zoom = 1.0, focal = g_pinhole_f, sensor_sz = g_pinhole_sensor_sz, img_sz = g_pihole_img_sz):
    return frender_verts(fread_obj_vertices_file(filename), adjust, adjust_zoom, focal, sensor_sz, img_sz)
#ftest_obj()

In [20]:
#fimg_ar_disp(frender_model_img())

In [13]:
import math

def faxisang_mat(axis, theta):
    axis = np.asarray(axis)
    axis = axis/math.sqrt(np.dot(axis, axis))
    a = math.cos(theta/2.0)
    b, c, d = -axis*math.sin(theta/2.0)
    aa, bb, cc, dd = a*a, b*b, c*c, d*d
    bc, ad, ac, ab, bd, cd = b*c, a*d, a*c, a*b, b*d, c*d
    return np.array([[aa+bb-cc-dd, 2*(bc+ad), 2*(bd-ac)],
                     [2*(bc-ad), aa+cc-bb-dd, 2*(cd+ab)],
                     [2*(bd+ac), 2*(cd-ab), aa+dd-bb-cc]])

In [35]:
def fstrip_gen_verts(dimx=0, dimy=1, w=0.15, ry=0.3):
    verts = np.zeros((4,3))
    verts[0][dimx] = -w/2; verts[1][dimx] = w/2; verts[2][dimx] = -w/2; verts[3][dimx] = w/2;
    verts[0][dimy] = 0.0; verts[1][dimy] = 0.0; verts[2][dimy] = ry; verts[3][dimy] = ry;
    return verts
#fimg_ar_disp(frender_verts_img(fstrip_gen_verts(), adjust = True, adjust_zoom = 3.0))

def test_strip_render_anim():
    orig_verts = fstrip_gen_verts()
    axis = [1,0,0]
    imgs = []
    for th in range(0, 90, 10):
        mat = faxisang_mat(axis, th*np.pi/180)
        verts = [np.dot(mat, x) for x in orig_verts]
        imgs.append(frender_verts_img(verts, adjust = True, adjust_zoom = 3.0))
        #fimg_ar_disp(imgs[-1])
    return fimg_animate_plt(imgs)
    
#HTML(test_strip_render_anim().to_html5_video())